<a href="https://colab.research.google.com/github/Tchy258/Proyectos-discretas/blob/main/Proyecto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pysat
!pip install folium
!pip install python-sat[pblib,aiger]
!pip install python-sat
!pip install -U python-sat

In [ ]:
import random
import numpy as np
from pysat.solvers import Minisat22
import math

#Primero hay que ver cuantas n clausulas hay que recibir (cuantos array hay que recibir)

cantidadClausulas=int(input('Cantidad de clausulas: '))

#Luego hay que ver cuantas proposiciones hay

cantidadProposiciones=int(input('Cantidad de proposiciones'))

#Hay que generar aleatoriedad

#negadorAleatorio recibe una proposicion x, y dependiendo del valor de la variable negar, la niega o la mantiene
def negadorAleatorio(x):
    negar=random.random() #negar es un numero aleatorio entre 0 y 1
    if negar>=0.5: #Si negar es mayor que 0.5
        return x #La proposición se mantiene
    else: #Si no
        return -x #Se niega

#generarListaAleatoria recibe la cantidad de clausulas y la cantidad de proposiciones y genera una lista al azar de formulas en 3-CNF
def generarListaAleatoria(clausulas,proposiciones):
    listaGenerada=np.zeros(clausulas,3) #Primero se crea una matriz de nx3 donde n es la cantidad de clausulas (o conjunciones)
    for i in range(0,clausulas): #Por cada clausula
        for j in range(0,3): #Para cada proposición dentro de una clausula
            proposicion=negadorAleatorio(random.randint(0,proposiciones)) #Elegir un literal al azar y al mismo tiempo negarlo aleatoriamente
            listaGenerada[i][j]=proposicion #Agregar el literal aleatorizado a la clausula i en el espacio j de la disyunción
    return listaGenerada


#valuacionObjetivo devuelve la combinacion de literales que satisface una disyunción
def valuacionObjetivo(literales):
  objetivo=np.full(3,-1) #objetivo es un array que determina el objetivo de valores de verdad segun el signo de los literales
  for i in range (0,literales):
    if literales[i]>0:
      objetivo[i]=1 #1 si es que el literal no esta negado
    else:
      objetivo[i]=0 #0 si es que está negado
  return objetivo

#sigmaGorro devuelve la valuación de una disyunción de literales
def sigmaGorro(valores,objetivo):
  objetivo=np.full(3,-1)
  for i in range (0,valores):
    if valores[i]==objetivo[i]:
      pass #Si el valor i de la fila de la tabla de verdad corresponde al objetivo, se continúa evaluando
    else:
      return False #Si no corresponde, esta fila de combinaciones no satisface la disyunción
  return True #Si termina el for loop, todo coincide, por lo que la combinación si satisface la disyunción


#buscarValuacionDesde busca una valuacion de tabla de verdad a partir de una cierta fila, devuelve el indice de la primera verdadera, o -1 si es que no hay
def buscarValuacionDesde(indice,valuaciones):
  if indice<8:
    for i in range(indice,8):
      if valuaciones[i]==True:
        return i
  return -1

#asignarValoresDeVerdad coloca el valor correspondiente según dice la tabla cada proposición de la clausula a revisar y luego retorna el arreglo
#de valores junto con un valor de True o False donde True significa que no hay contradicciones y False significa que si
def asignarValoresDeVerdad(filaDeTabla,clausula,valoresDeVerdad):
  literal1=int(abs(clausula[0])) - 1
  literal2=int(abs(clausula[1])) - 1
  literal3=int(abs(clausula[2])) - 1
  if filaDeTabla==0:
    if valoresDeVerdad[literal1]==-1:
      valoresDeVerdad[literal1]=1
    elif valoresDeVerdad[literal1]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      valoresDeVerdad[literal2]=1
    elif valoresDeVerdad[literal2]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      valoresDeVerdad[literal3]=1
    elif valoresDeVerdad[literal3]==0:
      return (valoresDeVerdad,False)
  if filaDeTabla==1:
    if valoresDeVerdad[literal1]==-1:
      valoresDeVerdad[literal1]=1
    elif valoresDeVerdad[literal1]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      valoresDeVerdad[literal2]=1
    elif valoresDeVerdad[literal2]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      valoresDeVerdad[literal3]=0
    elif valoresDeVerdad[literal3]==1:
      return (valoresDeVerdad,False)
  if filaDeTabla==2:
    if valoresDeVerdad[literal1]==-1:
      valoresDeVerdad[literal1]=1
    elif valoresDeVerdad[literal1]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      valoresDeVerdad[literal2]=0
    elif valoresDeVerdad[literal2]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      valoresDeVerdad[literal3]=1
    elif valoresDeVerdad[literal3]==0:
      return (valoresDeVerdad,False)
  if filaDeTabla==3:
    if valoresDeVerdad[literal1]==-1:
      valoresDeVerdad[literal1]=1
    elif valoresDeVerdad[literal1]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      valoresDeVerdad[literal2]=0
    elif valoresDeVerdad[literal2]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      valoresDeVerdad[literal3]=0
    elif valoresDeVerdad[literal3]==1:
      return (valoresDeVerdad,False)
  if filaDeTabla==4:
    if valoresDeVerdad[literal1]==-1:
      valoresDeVerdad[literal1]=0
    elif valoresDeVerdad[literal1]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      valoresDeVerdad[literal2]=1
    elif valoresDeVerdad[literal2]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      valoresDeVerdad[literal3]=1
    elif valoresDeVerdad[literal3]==0:
      return (valoresDeVerdad,False)
  if filaDeTabla==5:
    if valoresDeVerdad[literal1]==-1:
      valoresDeVerdad[literal1]=0
    elif valoresDeVerdad[literal1]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      valoresDeVerdad[literal2]=1
    elif valoresDeVerdad[literal2]==0:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      valoresDeVerdad[literal3]=0
    elif valoresDeVerdad[literal3]==1:
      return (valoresDeVerdad,False)
  if filaDeTabla==6:
    if valoresDeVerdad[literal1]==-1:
      valoresDeVerdad[literal1]=0
    elif valoresDeVerdad[literal1]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      valoresDeVerdad[literal2]=0
    elif valoresDeVerdad[literal2]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      valoresDeVerdad[literal3]=1
    elif valoresDeVerdad[literal3]==0:
      return (valoresDeVerdad,False)
  if filaDeTabla==7:
    if valoresDeVerdad[literal1]==-1:
      valoresDeVerdad[literal1]=0
    elif valoresDeVerdad[literal1]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal2]==-1:
      valoresDeVerdad[literal2]=0
    elif valoresDeVerdad[literal2]==1:
      return (valoresDeVerdad,False)
    if valoresDeVerdad[literal3]==-1:
      valoresDeVerdad[literal3]=0
    elif valoresDeVerdad[literal3]==1:
      return (valoresDeVerdad,False)
    return (valoresDeVerdad,True)
  

def recorrido(camino,actual,anterior,valoresDeVerdad,combinacionesSatisfactorias):






#evaluarLista recibe una lista de literales en 3-CNF y dice si es satisfactible o no y si es que si, dice cual combinación la satisface.
def evaluarLista(lista,clausulas,proposiciones):
  #subTabla es una tabla de verdad que va a ser utilizada por cada clausula 3-CNF
  subTabla=np.array([1,1,1],[1,1,0],[1,0,1],[1,0,0],[0,1,1],[0,1,0],[0,0,1],[0,0,0])
  combinacionesSatisfactorias=np.full((clausulas,8),False) #Registro de cuales combinaciones satisfacen la disyunción de literales por cada clausula
  cantidadDeCombinacionesPosibles=0
  for i in range(0,clausulas): #Por cada conjunción
    objetivo=valuacionObjetivo(lista[i]) #Ver la valuación objetivo
    for j in range(0,8): #Por cada combinación de valores de la tabla de verdad
      satisface=sigmaGorro(subTabla[j],lista[i]) #Ver si corresponde
      if satisface: #Si corresponde
        combinacionesSatisfactorias[i][j]=True #Registrarla
  caminosViables=np.full((clausulas,8),(-1,-1)) #Registro de caminos posibles, -1 es desconocido, 1 es viable y 0 es inviable, el segundo numero indica el origen del camino
  #-1 es el valor por defecto y si el camino es viable, este valor cambia al numero de la fila de la tabla de verdad de la clausula anterior
  inicio=0 #Indice para elegir el inicio del camino (la combinacion de la subtabla de verdad de la primera combinación que no ha sido revisada)
  while inicio < 8: #Mientras no haya revisado todos los inicios de camino posibles
    valoresDeVerdad=np.full(proposiciones,-1) #Registro de valores de verdad para cada literal para evaluar cada combinación de combinaciones posible y encontrar una satisfactoria
    indiceSubtabla1=buscarValuacionDesde(inicio,combinacionesSatisfactorias[0]) #Elegir una combinación de la subtabla de verdad de la clausula inicial a partir del indice inicio
    caminoAnterior=-1
    for i in range(0,clausulas): #Por cada clausula
      if i==0: #Si estoy en la primera clausula
        disyuncion=combinacionesSatisfactorias[i][indiceSubtabla1] #Usar el inicio del camino
        (valoresDeVerdad,satisfactible)=asignarValoresDeVerdad(disyuncion,clausula[i],valoresDeVerdad) #Actualizar valores de verdad
        caminoAnterior=disyuncion #Recordar desde donde vengo
      else: #Si no estoy en la primera
        for j in range(0,8): #Por cada camino posible
          indiceCamino=buscarValuacionDesde(j,combinacionesSatisfactorias[i]) #Buscar una combinación satisfactoria de la tabla de la clausula i
          disyuncion=combinacionesSatisfactorias[i][indiceCamino]
          (valoresDeVerdad,satisfactible)=asignarValoresDeVerdad(disyuncion,clausula[i],valoresDeVerdad)
          if not satisfactible:
            caminosViables[i][j]=(0,caminoAnterior)
          else:



      if disyuncion==-1: #Si no se encontró una valuacion de subtabla de verdad que satisface
        cantidadDeCombinacionesPosibles-=1 #Disminuir la cantidad de combinaciones en 1
        if i==0:
          return
      else:
        if caminosViables[i][indiceSubtabla]==-1: #Si este camino no ha sido revisado

          
        for j in clausulas[i]: #Por cada literal de la clausula i
          literal=int(abs(j)) #
          valoresDeVerdad[]
        
        
        
